In [ ]:
#! pip install keras==2.2.4

#! pip install innvestigate
%tensorflow_version 1.x
import keras
import keras.backend
import keras.layers
import keras.models
import keras.utils
import torch
import numpy as np
from google.colab import drive
import sys
import glob
import pickle
import scipy.io
import os
drive.mount("/content/drive/")
sys.path.append('/content/drive/My Drive/')

import innvestigate
import innvestigate.utils as iutils



TensorFlow 1.x selected.


Using TensorFlow backend.


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
class DEAP:
    def __init__(
        self, subject=12, num_segments=12, device="cpu", num_classes=3, upsample=True
    ):
        self.subject = subject
        self.num_segments = num_segments
        self.device = device
        self.num_classes = num_classes
        self.upsample = upsample
        self._load_data()

    def _get_subject_file_names(self, s):
        eeg_root_folder = "/content/drive/My Drive/data_preprocessed_matlab"
        landmarks_root_folder = "/content/drive/My Drive/landmarks"
        subject_code = "s" + "{:02d}".format(s)
        # Get eeg file name
        eeg_file = os.path.join(eeg_root_folder, subject_code + ".mat")
        # Get the landmark file names
        landmark_files = []
        for t in range(1, 41):
            trial_code = "trial" + "{:02d}".format(t)
            trial_file_name = os.path.join(
                landmarks_root_folder,
                subject_code,
                subject_code + "_" + trial_code + ".npy",
            )
            landmark_files.append(trial_file_name)
        return eeg_file, landmark_files

    def _read_eeg_file(self, file, seconds_to_remove=3, sampling_freq=128):
        data = scipy.io.loadmat(file)
        eeg_data = data["data"][
            :, :32, :
        ]  # First 32 channels belong to the EEG (video=40, channel=40, data=8064)
        eeg_data = eeg_data[
            :, :, sampling_freq * seconds_to_remove :
        ]  # Remove baseline recording
        labels = (
            data["labels"] // 3.3
        )  # valence, arousal, dominance, liking (video=40, 4)
        assert eeg_data.shape == (
            40,
            32,
            8064 - sampling_freq * seconds_to_remove,
        ), "EEG data reading failed!"
        assert labels.shape == (40, 4), "Labels reading failed!"
        return eeg_data, labels

    def _read_landmark_files(self, files):
        landmark_data = []
        for f in files:
            trial_data = np.load(f)
            trial_data = np.reshape(
                trial_data, (trial_data.shape[0], -1)
            )  # (3000, 81*2)
            trial_data = np.transpose(trial_data)  # (81*2, 3000)
            landmark_data.append(trial_data)
        landmark_data = np.array(
            landmark_data
        )  # (video=40, 81*2, 3000) Compatible with EEG
        assert landmark_data.shape == (
            40,
            81 * 2,
            3000,
        ), "Landmark data reading failed!"
        return landmark_data

    def _epoch_data(self, eeg_data, labels, landmark_data):
        assert (
            eeg_data.shape[0] == landmark_data.shape[0]
        ), "Mismatch between EEG and landmark data"
        assert (
            eeg_data.shape[2] % self.num_segments == 0
            and landmark_data.shape[2] % self.num_segments == 0
        ), "Choose a better number of segments"
        eeg_data_per_segment = int(eeg_data.shape[2] / self.num_segments)
        landmark_data_per_segment = int(landmark_data.shape[2] / self.num_segments)

        eeg, landmark = [], []
        for v in range(eeg_data.shape[0]):
            for i in range(self.num_segments):
                eeg.append(
                    eeg_data[
                        v, :, i * eeg_data_per_segment : (i + 1) * eeg_data_per_segment
                    ]
                )
                landmark.append(
                    landmark_data[
                        v,
                        :,
                        i
                        * landmark_data_per_segment : (i + 1)
                        * landmark_data_per_segment,
                    ]
                )

        self.eeg = np.array(eeg).transpose(0, 2, 1)  # (batch, seq, feature)
        self.landmark = np.array(landmark).transpose(0, 2, 1)  # (batch, seq, feature)
        self.labels = np.repeat(
            labels, self.num_segments, axis=0
        )  # (batch, 4 emotions)

        assert self.eeg.shape == (
            eeg_data.shape[0] * self.num_segments,
            eeg_data_per_segment,
            eeg_data.shape[1],
        ), "EEG epoching went wrong!"
        assert self.landmark.shape == (
            landmark_data.shape[0] * self.num_segments,
            landmark_data_per_segment,
            landmark_data.shape[1],
        ), "Landmark epoching went wrong"
        assert self.labels.shape == (
            labels.shape[0] * self.num_segments,
            4,
        ), "Label epoching went wrong"
        assert (
            self.eeg.shape[0] == self.landmark.shape[0]
            and self.landmark.shape[0] == self.labels.shape[0]
        ), "Number of data does not match!"

    def _load_data(self):
        eeg_file, landmark_files = self._get_subject_file_names(self.subject)
        eeg_data, labels = self._read_eeg_file(eeg_file)
        landmark_data = self._read_landmark_files(landmark_files)
        self._epoch_data(eeg_data, labels, landmark_data)
        if self.upsample:
            self._upsample_landmarks()
        self._normalize_landmarks()
        self._normalize_eeg()

    def _normalize_landmarks(self):
        mean = np.mean(self.landmark)
        std = np.std(self.landmark)
        self.landmark = (self.landmark - mean) / std

    def _normalize_eeg(self):
        mean = np.mean(self.eeg)
        std = np.std(self.eeg)
        self.eeg = (self.eeg - mean) / std

    def _upsample_landmarks(self):
        """
        Match the dimensions of landmarks and eeg data by linear upsampling
        """
        upsampler = torch.nn.Upsample(
            scale_factor=2.56, mode="linear", align_corners=False
        )
        self.landmark = upsampler(
            torch.from_numpy(self.landmark).permute(0, 2, 1)
        ).permute(
            0, 2, 1
        )  # batch, seq, feature
        self.landmark = self.landmark.numpy()

    def __len__(self):
        return self.eeg.shape[0]

    def __getitem__(self, ix):
        return {
            "eeg": self.eeg[ix],
            "face": self.landmark[ix],
            "label_val": keras.utils.to_categorical(
                self.labels[ix, 0], num_classes=self.num_classes
            ),
            "label_arousal": keras.utils.to_categorical(
                self.labels[ix, 1], num_classes=self.num_classes
            ),
        }

    def train_valid_split(self, split_ratio=0.2):
        split_idx = int(split_ratio * len(self))
        indices = np.arange(len(self))
        np.random.shuffle(indices)
        train_indices, val_indices = indices[split_idx:], indices[:split_idx]
        return train_indices, val_indices

In [ ]:
## Model collection
from innvestigate.utils.tests.networks import base as network_base
from keras.models import Model

def build_two_heads_net(input_shape_eeg=(None,1, 640, 32), input_shape_fl=(None, 1, 250, 81*2), output_n_eeg=8, output_n_fl=8, output_n=3):
    net = {}
    net["eeg_in"] = network_base.input_layer(shape=input_shape_eeg)
    net["conv1_eeg"] = keras.layers.Conv2D(32, (1,2), strides=(1, 1), activation="relu")(net["eeg_in"])
    net["conv2_eeg"] = keras.layers.Conv2D(64, (1, 2), strides=(1,1), activation="relu")(net["conv1_eeg"])
    net["pool_eeg"] = keras.layers.MaxPooling2D((1, 2))(net["conv2_eeg"])
    net["out_eeg"] = network_base.dense_layer(keras.layers.Flatten()(net["pool_eeg"]), units=output_n_eeg, activation='relu')
    net["fl_in"]=network_base.input_layer(shape=input_shape_fl)
    net["conv1_fl"] = keras.layers.Conv2D(32, (1, 2), strides=(1,1), activation="relu")(net["fl_in"])
    net["conv2_fl"] = keras.layers.Conv2D(64, (1, 2), strides=(1,1), activation="relu")(net["conv1_fl"])
    net["pool_fl"] = keras.layers.MaxPooling2D((1, 2))(net["conv2_fl"])
    net["out_fl"] = network_base.dense_layer(keras.layers.Flatten()(net["pool_fl"]), units=output_n_fl, activation='relu')
    net['concat_feat'] = keras.layers.merge.concatenate([net["out_eeg"], net["out_fl"]])
    net["concat_out"] = network_base.dense_layer(net["concat_feat"], units=32)
    net["out"] = network_base.dense_layer(net["concat_out"], units=output_n)
    net["sm_out"] = network_base.softmax(net["out"])

    model_with_softmax = Model(inputs=[net['eeg_in'], net['fl_in']], outputs=net['sm_out'])
    model_without_softmax = Model(inputs=[net['eeg_in'], net['fl_in']], outputs=net['out'])

    
    return model_with_softmax, model_without_softmax

def build_single_head_net(input_shape=(None, 1, 640, 81*2+32), feature_dim=16, output_n = 3):
    net = {}
    net["in"] = network_base.input_layer(shape=input_shape)
    net["conv1"] = keras.layers.Conv2D(filters=64, kernel_size=(1,2), strides=(1, 1), activation="relu" )(net["in"])
    net["conv2"] = keras.layers.Conv2D(filters=128, kernel_size=(1,2), activation="relu")(net["conv1"])
    net["pool"] = keras.layers.MaxPooling2D((1, 2))(net["conv2"])
    net["out_feat"] = network_base.dense_layer(keras.layers.Flatten()(net["pool"]), units=feature_dim, activation='relu')
    net["out"]= network_base.dense_layer(net["out_feat"], units=output_n)
    net["sm_out"] = network_base.softmax(net["out"])

    model_with_softmax = Model(inputs=net['in'], outputs=net['sm_out'])
    model_without_softmax = Model(inputs=net['in'], outputs=net['out'])

    return model_with_softmax, model_without_softmax

def eeg_net(input_shape_eeg=(None,1, 640, 32),output_n_eeg = 8, output_n = 3):
    net = {}
    net["in"] = network_base.input_layer(shape=input_shape_eeg)
    net["conv1_eeg"] = keras.layers.Conv2D(32, (1,2), strides=(1, 1), activation="relu")(net["in"])
    net["conv2_eeg"] = keras.layers.Conv2D(64, (1, 2), strides=(1,1), activation="relu")(net["conv1_eeg"])
    net["pool_eeg"] = keras.layers.MaxPooling2D((1, 2))(net["conv2_eeg"])
    net["out_eeg"] = network_base.dense_layer(keras.layers.Flatten()(net["pool_eeg"]), units=output_n_eeg, activation='relu')
    net["out"]= network_base.dense_layer(net["out_eeg"], units=output_n)
    net["sm_out"] = network_base.softmax(net["out"])

    model_with_softmax = Model(inputs=net['in'], outputs=net['sm_out'])
    model_without_softmax = Model(inputs=net['in'], outputs=net['out'])

    return model_with_softmax, model_without_softmax


def fl_net( input_shape_fl=(None, 1, 640, 81*2), output_n_fl = 8, output_n = 3):
    net = {}
    net["in"]=network_base.input_layer(shape=input_shape_fl)
    net["conv1_fl"] = keras.layers.Conv2D(32, (1, 2), strides=(1,1), activation="relu")(net["in"])
    net["conv2_fl"] = keras.layers.Conv2D(64, (1, 2), strides=(1,1), activation="relu")(net["conv1_fl"])
    net["pool_fl"] = keras.layers.MaxPooling2D((1, 2))(net["conv2_fl"])
    net["out_fl"] = network_base.dense_layer(keras.layers.Flatten()(net["pool_fl"]), units=output_n_fl, activation='relu')
    net["out"]= network_base.dense_layer(net["out_fl"], units=output_n)
    net["sm_out"] = network_base.softmax(net["out"])

    model_with_softmax = Model(inputs=net['in'], outputs=net['sm_out'])
    model_without_softmax = Model(inputs=net['in'], outputs=net['out'])

    return model_with_softmax, model_without_softmax
  
def pretrained(eeg_model, fl_model , input_shape_eeg = (None, 1, 640, 32), input_shape_fl= (None, 1, 640, 162)):
    net = {}
    for layer in eeg_model.layers[:-4]:
      layer.trainable = False
    for layer in fl_model.layers[:-4]:
      layer.trainable = False
    net['eeg_features'] = eeg_model.layers[-3].output
    net['fl_features'] = fl_model.layers[-3].output
    net['concat_feature']=keras.layers.merge.concatenate([net['eeg_features'], net['fl_features']])
    net['out1'] = network_base.dense_layer(net['concat_feature'], units=32)
    net['out'] = network_base.dense_layer(net['out1'], units =3)
                                          
    net["sm_out"] = network_base.softmax(net["out"])
    model_with_softmax = Model(inputs=[eeg_model.input, fl_model.input], outputs=net['sm_out'])

    return model_with_softmax



In [ ]:
## Single eeg or fl models
import random as rn
import tensorflow as tf
sd = 1 # Here sd means seed.
np.random.seed(sd)
rn.seed(sd)
os.environ['PYTHONHASHSEED']=str(sd)

from keras import backend as K
config = tf.ConfigProto(intra_op_parallelism_threads=1,inter_op_parallelism_threads=1)
tf.set_random_seed(sd)
sess = tf.Session(graph=tf.get_default_graph(), config=config)
K.set_session(sess)
## Load data
dataset=DEAP(subject=16, upsample = True )#
train_indices, valid_indices=dataset.train_valid_split()
train_X_eeg = dataset[train_indices]['eeg']
train_X_eeg = np.expand_dims(train_X_eeg, axis=1)
train_X_fl = dataset[train_indices]['face']
train_X_fl = np.expand_dims(train_X_fl, axis=1)
train_Y = dataset[train_indices]['label_arousal']
valid_X_eeg = dataset[valid_indices]['eeg']
valid_X_eeg = np.expand_dims(valid_X_eeg, axis=1)
valid_X_fl = dataset[valid_indices]['face']
valid_X_fl = np.expand_dims(valid_X_fl, axis=1)
valid_Y = dataset[valid_indices]['label_arousal']
keras.backend.image_data_format == "channels_first"

input_shape_fl=(1,640,162)
input_shape_eeg=(1,640,32)
concat_train = np.concatenate((train_X_eeg, train_X_fl), axis = -1)
concat_valid = np.concatenate((valid_X_eeg, valid_X_fl), axis = -1)
model_with_softmax, model_without_softmax=fl_net()
### eeg
eeg_model = keras.models.Sequential([
    keras.layers.Conv2D(32, (1, 2), strides=(1,1), activation="relu", input_shape=input_shape_eeg),
    keras.layers.Conv2D(64, (1, 2), activation="relu"),
    keras.layers.MaxPooling2D((1, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(8, activation="relu"),
    keras.layers.Dense(3, activation="softmax"),
])
###fl
fl_model = keras.models.Sequential([
    keras.layers.Conv2D(32, (1, 2), strides=(1,1), activation="relu", input_shape=input_shape_fl),
    keras.layers.Conv2D(64, (1, 2), activation="relu"),
    keras.layers.MaxPooling2D((1, 2)),
    keras.layers.Flatten(), 
    keras.layers.Dense(8, activation="relu"),
    keras.layers.Dense(3, activation="softmax"),
])

opt = keras.optimizers.Adam(lr=0.00005)
model_with_softmax.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
model_with_softmax.fit(train_X_fl, train_Y, epochs=50, batch_size=32,)
scores = model_with_softmax.evaluate(valid_X_fl, valid_Y, batch_size=32)
print("Scores on test set: loss=%s accuracy=%s" % tuple(scores))
model_without_softmax.set_weights(model_with_softmax.get_weights())

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:3103: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and now uses scale_factor directly, instead of relying on the computed output size. If you wish to restore the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor changed "


Epoch 1/50
384/384 [==============================] - 2s 6ms/step - loss: 0.8821 - accuracy: 0.6615
Epoch 2/50
384/384 [==============================] - 1s 3ms/step - loss: 0.8651 - accuracy: 0.6615
Epoch 3/50
384/384 [==============================] - 1s 3ms/step - loss: 0.8638 - accuracy: 0.6615
Epoch 4/50
384/384 [==============================] - 1s 3ms/step - loss: 0.8439 - accuracy: 0.6615
Epoch 5/50
384/384 [==============================] - 1s 3ms/step - loss: 0.8451 - accuracy: 0.6615
Epoch 6/50
384/384 [==============================] - 1s 3ms/step - loss: 0.8485 - accuracy: 0.6615
Epoch 7/50
384/384 [==============================] - 1s 3ms/step - loss: 0.8294 - accuracy: 0.6615
Epoch 8/50
384/384 [==============================] - 1s 3ms/step - loss: 0.8347 - accuracy: 0.6615
Epoch 9/50
384/384 [==============================] - 1s 3ms/step - loss: 0.8221 - accuracy: 0.6615
Epoch 10/50
384/384 [==============================] - 1s 3ms/step - loss: 0.8216 - accuracy: 0.6615

In [ ]:
model_with_softmax.save('/content/drive/My Drive/models/fl_softmax_subject_16.h5')

In [ ]:
## EEG+FL models 
import random as rn
import tensorflow as tf
sd = 1 # Here sd means seed.
np.random.seed(sd)
rn.seed(sd)
os.environ['PYTHONHASHSEED']=str(sd)

from keras import backend as K
config = tf.ConfigProto(intra_op_parallelism_threads=1,inter_op_parallelism_threads=1)
tf.set_random_seed(sd)
sess = tf.Session(graph=tf.get_default_graph(), config=config)
K.set_session(sess)
## Load data
concat = False
upsample = True
dataset=DEAP(subject=15, upsample = upsample)#
train_indices, valid_indices=dataset.train_valid_split()
train_X_eeg = dataset[train_indices]['eeg']
train_X_eeg = np.expand_dims(train_X_eeg, axis=1)
train_X_fl = dataset[train_indices]['face']
train_X_fl = np.expand_dims(train_X_fl, axis=1)
train_Y = dataset[train_indices]['label_arousal']
valid_X_eeg = dataset[valid_indices]['eeg']
valid_X_eeg = np.expand_dims(valid_X_eeg, axis=1)
valid_X_fl = dataset[valid_indices]['face']
valid_X_fl = np.expand_dims(valid_X_fl, axis=1)
valid_Y = dataset[valid_indices]['label_arousal']
keras.backend.image_data_format == "channels_first"

if concat:
  concat_train = np.concatenate((train_X_eeg, train_X_fl), axis = -1)
  concat_valid = np.concatenate((valid_X_eeg, valid_X_fl), axis = -1)
  model_with_softmax, model_without_softmax=build_single_head_net()
  opt = keras.optimizers.Adam(lr=0.00005)
  model_with_softmax.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
  model_with_softmax.fit(concat_train, train_Y, epochs=50, batch_size=32,)
  scores = model_with_softmax.evaluate(concat_valid, valid_Y, batch_size=32)
  print("Scores on test set: loss=%s accuracy=%s" % tuple(scores))
  model_without_softmax.set_weights(model_with_softmax.get_weights())

if not concat:
  opt = keras.optimizers.Adam(lr=0.00005)
  model_with_softmax, model_without_softmax=build_two_heads_net(input_shape_fl=(None, 1, 640, 81*2))
  model_with_softmax.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
  model_with_softmax.fit([train_X_eeg, train_X_fl], train_Y, epochs=50, batch_size=32,)
  scores = model_with_softmax.evaluate([valid_X_eeg, valid_X_fl], valid_Y, batch_size=32)
  print("Scores on test set: loss=%s accuracy=%s" % tuple(scores))
  model_without_softmax.set_weights(model_with_softmax.get_weights())

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:3103: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and now uses scale_factor directly, instead of relying on the computed output size. If you wish to restore the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor changed "


Epoch 1/50
384/384 [==============================] - 3s 9ms/step - loss: 1.0053 - accuracy: 0.5807
Epoch 2/50
384/384 [==============================] - 2s 5ms/step - loss: 0.8875 - accuracy: 0.6458
Epoch 3/50
384/384 [==============================] - 2s 5ms/step - loss: 0.8631 - accuracy: 0.6484
Epoch 4/50
384/384 [==============================] - 2s 5ms/step - loss: 0.8356 - accuracy: 0.6458
Epoch 5/50
384/384 [==============================] - 2s 5ms/step - loss: 0.8091 - accuracy: 0.6484
Epoch 6/50
384/384 [==============================] - 2s 5ms/step - loss: 0.7783 - accuracy: 0.6562
Epoch 7/50
384/384 [==============================] - 2s 5ms/step - loss: 0.7476 - accuracy: 0.6589
Epoch 8/50
384/384 [==============================] - 2s 5ms/step - loss: 0.7194 - accuracy: 0.6849
Epoch 9/50
384/384 [==============================] - 2s 5ms/step - loss: 0.6945 - accuracy: 0.6875
Epoch 10/50
384/384 [==============================] - 2s 5ms/step - loss: 0.6669 - accuracy: 0.6953

In [ ]:
## Pre-train + tuning model
## Single eeg or fl models
import random as rn
import tensorflow as tf
from keras import backend as K
sd = 1 # Here sd means seed.
np.random.seed(sd)
rn.seed(sd)
os.environ['PYTHONHASHSEED']=str(sd)
config = tf.ConfigProto(intra_op_parallelism_threads=1,inter_op_parallelism_threads=1)
tf.set_random_seed(sd)
sess = tf.Session(graph=tf.get_default_graph(), config=config)
K.set_session(sess)
# IMPORTANT: models have to be loaded AFTER SETTING THE SESSION for keras! 
# Otherwise, their weights will be unavailable in the threads after the session there has been set

eeg=keras.models.load_model('/content/drive/My Drive/models/eeg_softmax_subject_16.h5')
fl = keras.models.load_model('/content/drive/My Drive/models/fl_softmax_subject_16.h5')

concat = False
upsample = True
dataset=DEAP(subject=16, upsample = upsample)#
train_indices, valid_indices=dataset.train_valid_split()
train_X_eeg = dataset[train_indices]['eeg']
train_X_eeg = np.expand_dims(train_X_eeg, axis=1)
train_X_fl = dataset[train_indices]['face']
train_X_fl = np.expand_dims(train_X_fl, axis=1)
train_Y = dataset[train_indices]['label_arousal']
valid_X_eeg = dataset[valid_indices]['eeg']
valid_X_eeg = np.expand_dims(valid_X_eeg, axis=1)
valid_X_fl = dataset[valid_indices]['face']
valid_X_fl = np.expand_dims(valid_X_fl, axis=1)
valid_Y = dataset[valid_indices]['label_arousal']
keras.backend.image_data_format == "channels_first"

def pretrained(eeg_model, fl_model , input_shape_eeg = (None, 1, 640, 32), input_shape_fl= (None, 1, 640, 162)):
  net = {}
  for layer in eeg_model.layers[:-4]:
    layer.trainable = False
  for layer in fl_model.layers[:-4]:
    layer.trainable = False
  net['eeg_features'] = eeg_model.layers[-3].output
  net['fl_features'] = fl_model.layers[-3].output
  net['concat_feature']=keras.layers.merge.concatenate([net['eeg_features'], net['fl_features']])
  net['out1'] = network_base.dense_layer(net['concat_feature'], units=32)
  net['out'] = network_base.dense_layer(net['out1'], units =3)
                                        
  net["sm_out"] = network_base.softmax(net["out"])
  model_with_softmax = Model(inputs=[eeg_model.input, fl_model.input], outputs=net['sm_out'])

  return model_with_softmax

if concat:
  concat_train = np.concatenate((train_X_eeg, train_X_fl), axis = -1)
  concat_valid = np.concatenate((valid_X_eeg, valid_X_fl), axis = -1)
  model_with_softmax, model_without_softmax=build_single_head_net()
  opt = keras.optimizers.Adam(lr=0.00005)
  model_with_softmax.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
  model_with_softmax.fit(concat_train, train_Y, epochs=50, batch_size=32,)
  scores = model_with_softmax.evaluate(concat_valid, valid_Y, batch_size=32)
  print("Scores on test set: loss=%s accuracy=%s" % tuple(scores))
  model_without_softmax.set_weights(model_with_softmax.get_weights())

if not concat:
  model_with_softmax=pretrained(eeg, fl)
  opt = keras.optimizers.Adam(lr=0.00005)
  #model_with_softmax, model_without_softmax=build_two_heads_net(input_shape_fl=(None, 1, 640, 81*2))
  model_with_softmax.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
  model_with_softmax.fit([train_X_eeg, train_X_fl], train_Y, epochs=50, batch_size=32,)
  scores = model_with_softmax.evaluate([valid_X_eeg, valid_X_fl], valid_Y, batch_size=32)
  print("Scores on test set: loss=%s accuracy=%s" % tuple(scores))
  #model_without_softmax.set_weights(model_with_softmax.get_weights())

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:3103: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and now uses scale_factor directly, instead of relying on the computed output size. If you wish to restore the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor changed "


Epoch 1/50
384/384 [==============================] - 1s 3ms/step - loss: 1.1031 - accuracy: 0.4974
Epoch 2/50
384/384 [==============================] - 1s 2ms/step - loss: 0.8409 - accuracy: 0.6536
Epoch 3/50
384/384 [==============================] - 1s 2ms/step - loss: 0.8088 - accuracy: 0.6562
Epoch 4/50
384/384 [==============================] - 1s 2ms/step - loss: 0.7884 - accuracy: 0.6641
Epoch 5/50
384/384 [==============================] - 1s 2ms/step - loss: 0.7868 - accuracy: 0.6615
Epoch 6/50
384/384 [==============================] - 1s 2ms/step - loss: 0.7853 - accuracy: 0.6589
Epoch 7/50
384/384 [==============================] - 1s 2ms/step - loss: 0.7855 - accuracy: 0.6615
Epoch 8/50
384/384 [==============================] - 1s 2ms/step - loss: 0.7801 - accuracy: 0.6641
Epoch 9/50
384/384 [==============================] - 1s 2ms/step - loss: 0.7811 - accuracy: 0.6615
Epoch 10/50
384/384 [==============================] - 1s 2ms/step - loss: 0.7777 - accuracy: 0.6615

In [ ]:
model_with_softmax.save('/content/drive/My Drive/models/pretrain_softmax_subject_15-2.h5')

# New section